In [1]:
#from google.colab import files
import pandas as pd
import numpy as np
import calendar
# import warnings

import matplotlib.pyplot as plt

In [2]:
# Cargamos el archivo Excel - RFF Procesado
# --------------

ruta_path = 'C:/cesar_quezada/ProyectosIA/Proyectos_PALMAS'               # -> Casa
# ruta_path = 'C:/Users/usuario/Documents/DOCUMENTOS_CONSULTOR_5_BI'      # -> Trabajo

path_perdidas_aceite_pw = '/PROYECTO_01_analitica_descriptiva_TEA/data/external/bbdd_perdidas_aceite/704d248b-f0e9-441e-a80e-pw.xlsx'
path_perdidas_aceite_nh = '/PROYECTO_01_analitica_descriptiva_TEA/data/external/bbdd_perdidas_aceite/0b141767-ddcb-4cb5-ac23-nh.xlsx'
path_perdidas_aceite_sh = '/PROYECTO_01_analitica_descriptiva_TEA/data/external/bbdd_perdidas_aceite/ceceab95-d698-4ca2-a7dc-sh.xlsx'
path_perdidas_aceite_tl = '/PROYECTO_01_analitica_descriptiva_TEA/data/external/bbdd_perdidas_aceite/3d4df57c-0e6a-45ac-b670-tl.xlsx'


# Cargar la hoja 2 (indexada desde 0, por lo tanto la hoja 2 es 'sheet_name=1')
df_perdida_aceite_pw = pd.read_excel(ruta_path + path_perdidas_aceite_pw,sheet_name='Sheet1')
df_perdida_aceite_nh = pd.read_excel(ruta_path + path_perdidas_aceite_nh,sheet_name='Sheet1')
df_perdida_aceite_sh = pd.read_excel(ruta_path + path_perdidas_aceite_sh,sheet_name='Sheet1')
df_perdida_aceite_tl = pd.read_excel(ruta_path + path_perdidas_aceite_tl,sheet_name='Sheet1')

df_perdida_aceite_pw = df_perdida_aceite_pw[~df_perdida_aceite_pw[["AÑO", "MES", "DIA"]].isin(['Totales']).any(axis=1) & df_perdida_aceite_pw[["AÑO", "MES", "DIA"]].notna().all(axis=1)]
df_perdida_aceite_nh = df_perdida_aceite_nh[~df_perdida_aceite_nh[["AÑO", "MES", "DIA"]].isin(['Totales']).any(axis=1) & df_perdida_aceite_nh[["AÑO", "MES", "DIA"]].notna().all(axis=1)]
df_perdida_aceite_sh = df_perdida_aceite_sh[~df_perdida_aceite_sh[["AÑO", "MES", "DIA"]].isin(['Totales']).any(axis=1) & df_perdida_aceite_sh[["AÑO", "MES", "DIA"]].notna().all(axis=1)]
df_perdida_aceite_tl = df_perdida_aceite_tl[~df_perdida_aceite_tl[["AÑO", "MES", "DIA"]].isin(['Totales']).any(axis=1) & df_perdida_aceite_tl[["AÑO", "MES", "DIA"]].notna().all(axis=1)]

df_perdida_aceite_nh['PLANTACION'] = 'Nuevo Horizonte'
df_perdida_aceite_pw['PLANTACION'] = 'Palmawasi'
df_perdida_aceite_sh['PLANTACION'] = 'Shanuzi'
df_perdida_aceite_tl['PLANTACION'] = 'Tulumayo'

# Concatenar los DataFrames
df_perdida_aceite_full = pd.concat([df_perdida_aceite_nh, df_perdida_aceite_pw, df_perdida_aceite_sh, df_perdida_aceite_tl], ignore_index=True)

# Creamos variable fecha
df_perdida_aceite_full['FECHA'] = pd.to_datetime(df_perdida_aceite_full['DIA'])


print(df_perdida_aceite_full.dtypes)
df_perdida_aceite_full.head()


AÑO                                                int64
MES                                               object
DIA                                               object
RFF PROCESADO                                    float64
TM CONDENSADO                                    float64
TM ACEITE CONDENSADO                             float64
% PERDIDA ACEITE CONDENSADO                       object
TM ESCOBAJO SIN PICAR                            float64
TM ACEITE ESCOBAJO SIN PICAR                     float64
% PERDIDA ACEITE ESCOBAJO SIN PICAR               object
TM ESCOBAJO PICADO                               float64
TM ACEITE ESCOBAJO PICADO                        float64
% PERDIDA ACEITE ESCOBAJO PICADO                  object
% PERDIDA TOTAL ACEITE ESCOBAJO                   object
% RACIMOS DUROS                                   object
% RACIMOS MAL DESFRUTADO ANORMAL                  object
% RACIMO MAL DESFRUTADO                           object
FACTOR                         

,AÑO,MES,DIA,RFF PROCESADO,TM CONDENSADO,TM ACEITE CONDENSADO,% PERDIDA ACEITE CONDENSADO,TM ESCOBAJO SIN PICAR,TM ACEITE ESCOBAJO SIN PICAR,% PERDIDA ACEITE ESCOBAJO SIN PICAR,...,% PERDIDA ACEITE SOLIDO TD,TM ARENA,TM ACEITE EN ARENA,% PERDIDA ACEITE EN ARENA,TM NUEZ,TM ACEITE CASCARA,% PERDIDA ACEITE EN CASCARA,% PERDIDA TOTAL,PLANTACION,FECHA
0,2024,Enero,2024-01-01 00:00:00,0.00000,0.000000,0.000000,-,0.000000,0.000000,-,...,-,0.000000,0.000000,-,0.000000,0.000000,-,-,Nuevo Horizonte,2024-01-01
1,2024,Enero,2024-01-02 00:00:00,302.26061,19.042418,0.238901,0.00079,52.895607,2.382416,0.007882,...,0.002221,0.634747,0.057962,0.000192,21.158243,0.106710,0.000353,0.020257,Nuevo Horizonte,2024-01-02
2,2024,Enero,2024-01-03 00:00:00,338.41343,21.320046,0.258341,0.000763,59.222350,3.375230,0.009974,...,0.002029,0.710668,0.027868,0.000082,23.688940,0.165019,0.000488,0.026964,Nuevo Horizonte,2024-01-03
3,2024,Enero,2024-01-04 00:00:00,236.91708,14.925776,0.070097,0.000296,41.460489,2.362937,0.009974,...,0.001734,0.497526,0.033217,0.00014,16.584196,0.107534,0.000454,0.022926,Nuevo Horizonte,2024-01-04
4,2024,Enero,2024-01-05 00:00:00,264.46554,16.661329,0.075064,0.000284,46.281470,2.637697,0.009974,...,0.001735,0.555378,0.067796,0.000256,18.512588,0.124013,0.000469,0.024198,Nuevo Horizonte,2024-01-05


### Cargando datos de PERDIDAS EN ACEITE

In [3]:
# Guardamos la data en processed
# -----
# Especifica la ruta donde deseas guardar el CSV
ruta_perdida_aceite_full =  '/PROYECTO_01_analitica_descriptiva_TEA/data/processed/df_perdida_aceite_full.csv'  # Cambia esta ruta a la que necesites
# Guardar el DataFrame en formato CSV
df_perdida_aceite_full.to_csv(ruta_path + ruta_perdida_aceite_full, index=False)
print(f'Datos guardados en {ruta_path + ruta_perdida_aceite_full}')

df_perdida_aceite_full.tail(10)

Datos guardados en C:/cesar_quezada/ProyectosIA/Proyectos_PALMAS/PROYECTO_01_analitica_descriptiva_TEA/data/processed/df_perdida_aceite_full.csv


,AÑO,MES,DIA,RFF PROCESADO,TM CONDENSADO,TM ACEITE CONDENSADO,% PERDIDA ACEITE CONDENSADO,TM ESCOBAJO SIN PICAR,TM ACEITE ESCOBAJO SIN PICAR,% PERDIDA ACEITE ESCOBAJO SIN PICAR,...,% PERDIDA ACEITE SOLIDO TD,TM ARENA,TM ACEITE EN ARENA,% PERDIDA ACEITE EN ARENA,TM NUEZ,TM ACEITE CASCARA,% PERDIDA ACEITE EN CASCARA,% PERDIDA TOTAL,PLANTACION,FECHA
3840,2022,Diciembre,2022-12-22 00:00:00,472.87,0.0,0.0,0,0.0,0.0,0,...,0,0.004101,0.0,0,33.1009,0.0,0,0,Tulumayo,2022-12-22
3841,2022,Diciembre,2022-12-23 00:00:00,512.70,0.0,0.0,0,0.0,0.0,0,...,0,0.004447,0.0,0,35.8890,0.0,0,0,Tulumayo,2022-12-23
3842,2022,Diciembre,2022-12-24 00:00:00,300.28,0.0,0.0,0,0.0,0.0,0,...,0,0.002604,0.0,0,21.0196,0.0,0,0,Tulumayo,2022-12-24
3843,2022,Diciembre,2022-12-25 00:00:00,0.00,0.0,0.0,-,0.0,0.0,-,...,-,0.000000,0.0,-,0.0000,0.0,-,-,Tulumayo,2022-12-25
3844,2022,Diciembre,2022-12-26 00:00:00,307.18,0.0,0.0,0,0.0,0.0,0,...,0,0.002664,0.0,0,21.5026,0.0,0,0,Tulumayo,2022-12-26
3845,2022,Diciembre,2022-12-27 00:00:00,359.27,0.0,0.0,0,0.0,0.0,0,...,0,0.003116,0.0,0,25.1489,0.0,0,0,Tulumayo,2022-12-27
3846,2022,Diciembre,2022-12-28 00:00:00,414.11,0.0,0.0,0,0.0,0.0,0,...,0,0.003592,0.0,0,28.9877,0.0,0,0,Tulumayo,2022-12-28
3847,2022,Diciembre,2022-12-29 00:00:00,530.50,0.0,0.0,0,0.0,0.0,0,...,0,0.004601,0.0,0,37.1350,0.0,0,0,Tulumayo,2022-12-29
3848,2022,Diciembre,2022-12-30 00:00:00,608.20,0.0,0.0,0,0.0,0.0,0,...,0,0.005275,0.0,0,42.5740,0.0,0,0,Tulumayo,2022-12-30
3849,2022,Diciembre,2022-12-31 00:00:00,318.68,0.0,0.0,0,0.0,0.0,0,...,0,0.002764,0.0,0,22.3076,0.0,0,0,Tulumayo,2022-12-31
